# Launch object detection *training*

## Installation

You should select a runtime with GPU first: (Runtime -> Change Runtime Type = GPU)

In [0]:
!apt-get install libsm6 libxext6 libxrender-dev libcap-dev ffmpeg
!pip install --upgrade git+https://github.com/surfriderfoundationeurope/mot.git
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

## Training

### Downloading dataset

In [0]:
import wget
from zipfile import ZipFile

wget.download("http://files.heuritech.com/raw_files/dataset_surfrider_cleaned.zip", ".")

with ZipFile('dataset_surfrider_cleaned.zip', 'r') as zipObj:
     zipObj.extractall()

### Download pretrained weights on ImageNet

In [0]:
wget.download("http://models.tensorpack.com/FasterRCNN/COCO-MaskRCNN-R50FPN2x.npz", ".")

### Launch the training

See this [file](https://github.com/surfriderfoundationeurope/mot/blob/master/src/mot/object_detection/README.md) to choose the architecture you want and the according pre trained weights. The weights downloaded above correspond to ResNet50-FPN with 2X scheduling.  
Since we are not interested in segmentation, we set MODE_MASK=False

In [0]:
!git clone https://github.com/surfriderfoundationeurope/mot.git

In [0]:
import sys

sys.path.append("mot/src")

In [0]:
!python3 mot/src/mot/object_detection/train.py --load COCO-MaskRCNN-R50FPN2x.npz --logdir model --config DATA.BASEDIR=dataset_surfrider_cleaned MODE_MASK=False

## Visualize dataset

In [0]:
from mot.object_detection.viz import draw_annotation
from mot.object_detection.dataset.mot import MotDataset
from mot.object_detection.dataset import register_mot
from mot.object_detection.config import finalize_configs
from mot.object_detection.config import config as cfg


cfg.DATA.BASEDIR = "dataset_surfrider_cleaned"
register_mot(cfg.DATA.BASEDIR)
finalize_configs(is_training=False)

In [0]:
dataset = MotDataset(base_dir="dataset_surfrider_cleaned/", split="val")
roids = dataset.training_roidbs()

In [0]:
from google.colab.patches import cv2_imshow
import cv2
for r in dataset.training_roidbs():
    im = cv2.imread(r["file_name"])
    vis = draw_annotation(im, r["boxes"], r["class"])
    cv2_imshow(vis)